# Ceneo Scraper

## Loading libraries

In [11]:
import requests
from bs4 import BeautifulSoup

## Sending request to Ceneo.pl server

In [10]:
product_id = "44958016"
url = f"https://www.ceneo.pl/{product_id}#tab=reviews"
response = requests.get(url)




<!DOCTYPE html>
<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7  non-direct ab-inactive" lang="pl"> <![endif]-->
<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8  non-direct ab-inactive" lang="pl"> <![endif]-->
<!--[if IE 8]>         <html class="no-js lt-ie9  non-direct ab-inactive" lang="pl"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="no-js  non-direct ab-inactive" lang="pl">
<!--<![endif]-->
<head>
    <meta charset="utf-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <link rel="preconnect" href="https://image.ceneo.pl">
<link rel="preconnect" href="https://image.ceneostatic.pl">
<link rel="preconnect" href="https://www.google-analytics.com">
<link rel="preconnect" href="https://www.googleadservices.com">
<link rel="preconnect" href="https://adservice.google.com">
<link rel="preconnect" href="https://www.google.pl">
<link rel="preconnect" href="https://www.google.com">
<link rel="preconnect" href="https://cdn.ampproject.org">

    

  

In [14]:
page_dom = BeautifulSoup(response.text, "html.parser")
opinions = page_dom.select("div.js_product-review")
opinion = page_dom.select_one("div.js_product-review")
print(type(page_dom))
print(type(opinions))
print(type(opinion))
print(opinion)

<class 'bs4.BeautifulSoup'>
<class 'bs4.element.ResultSet'>
<class 'bs4.element.Tag'>
<div class="user-post user-post__card js_product-review" data-entry-id="4007278">
<header class="user-post__header">
<div class="js_lazy user-post__avatar user-rank__avatar" data-bg="/Content/img/account/avatar/1.svg"></div>
</header>
<div class="user-post__body">
<div class="user-post__content">
<span class="user-post__author-name">
krzysiek</span>
<span class="user-post__author-recomendation">
<em class="recommended">Polecam</em>
</span>
<span class="user-post__score">
<span class="screen-reader-text">Ocena:</span>
<span class="score-container score-container--s js_score-container">
<span class="score-marker score-marker--s" style="width: 90.00%;"></span>
</span>
<span class="user-post__score-count">4,5/5</span>
<span class="user-post__published">
        Wystawiono
        <time datetime="2016-10-04 14:04:57">8 lat temu, </time>
<time datetime="2016-09-20 16:02:16">po 2 tygodniach</time> użytkowani

## Extraction of components of single opinion

|Component|Selector|Variable|
|---------|--------|--------|
|opinion ID| | |
|opinion’s author| | |
|author’s recommendation| | |
|score expressed in number of stars| | |
|opinion’s content| | |
|list of product advantages| | |
|list of product disadvantages| |  |
|how many users think that opinion was helpful| | |
|how many users think that opinion was unhelpful| | |
|publishing date| | |
|purchase date| | |
